In [5]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split

# Define paths
data_path = 'Cashew Kernel Classification.v2i.multiclass/Train_data/train'

# Define class labels
class_labels = {'W180': 0, 'W320': 1, 'W400': 2}

# Custom function to load and label images
def load_and_label_images(data_path, class_labels):
    images = []
    labels = []
    for filename in os.listdir(data_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img_path = os.path.join(data_path, filename)
            img = load_img(img_path, target_size=(224, 224))
            img_array = img_to_array(img) / 255.0
            images.append(img_array)
            label_str = filename[:4]
            label = class_labels.get(label_str)
            labels.append(label)
    return np.array(images), np.array(labels)

# Load and preprocess images
images, labels = load_and_label_images(data_path, class_labels)
labels = to_categorical(labels, num_classes=3)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes for W180, W320, W400
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callback functions for better training management
checkpoint = ModelCheckpoint('my_model.keras', save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,
    validation_data=(X_val, y_val),
    callbacks=[checkpoint, early_stopping],
    batch_size=16
)


C:\Users\EXAM-OS\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 113s 530ms/step - accuracy: 0.9138 - loss: 0.2426 - val_accuracy: 1.0000 - val_loss: 2.8193e-06
Epoch 2/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 103s 495ms/step - accuracy: 0.9975 - loss: 0.0108 - val_accuracy: 1.0000 - val_loss: 2.5410e-06
Epoch 3/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 101s 486ms/step - accuracy: 1.0000 - loss: 3.3768e-05 - val_accuracy: 1.0000 - val_loss: 2.4028e-07
Epoch 4/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 102s 491ms/step - accuracy: 1.0000 - loss: 2.9658e-05 - val_accuracy: 1.0000 - val_loss: 2.2925e-08
Epoch 5/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 104s 499ms/step - accuracy: 1.0000 - loss: 3.2121e-05 - val_accuracy: 1.0000 - val_loss: 1.0173e-08
Epoch 6/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 105s 505ms/step - accuracy: 1.0000 - loss: 1.6346e-06 - val_accuracy: 1.0000 - val_loss: 4.0119e-09
Epoch 7/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 105s 503ms/step - accuracy: 1.0000 - loss: 2.3734e-06 - val_accuracy: 1.0000 - val_loss: 1.7194e-09
Epoch 8/20
208/208 ━━━━━━━━━

In [14]:
from tensorflow.keras.preprocessing import image

def classify_image(model, img_path, threshold=0.7):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    max_prob = np.max(predictions)
    if max_prob < threshold:
        return 'Grade_D'
    else:
        class_idx = np.argmax(predictions)
        classes = ['W180', 'W320', 'W400']
        return classes[class_idx]

# Example usage
img_path = 'nature.jpg'
result = classify_image(model, img_path)
print(f'The image is classified as: {result}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
The image is classified as: W180
